## Transfer Learning
*This notebook is for creating a model to predict the crop diseases using transfer learning*

Steps to build our model.

1.   Image Preprocessing
      * Convert to array

2.   Image Augmentation using Keras Image Data Generator
3.   Model Building
      * Use Transfer Learning models like VGG16, Xception, MobileVNet, etc.

*Create a function that will allow us to run the model over and over again to check for optimal parameters*

## Image Preprocessing

In [ ]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator

In [ ]:
def preprocess(image):
  img = cv2.GaussianBlur(image,(5,5),0)
  return img_to_array(img)

## Image Augmentation

In [ ]:
# VARIABLES

TARGET_SIZE = (224, 224)
BATCH_SIZE = 50
EPOCHS = 40

LEARNING_RATE = 1e-4

In [ ]:
train_path = '/content/drive/MyDrive/Omdena/Osun Nigeria/Data/Maize Images/data/train'
test_path = '/content/drive/MyDrive/Omdena/Osun Nigeria/Data/Maize Images/data/test'

In [ ]:
train_generator = ImageDataGenerator(rescale=1.0/255,
                             rotation_range=30,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.2,
                             shear_range=0.3,
                             horizontal_flip=True,
                             brightness_range=[0.2, 0.8],
                             validation_split=0.2,
                             preprocessing_function=preprocess)

valid_generator = ImageDataGenerator(rescale=1./255,
                                     validation_split = 0.2,
                                   preprocessing_function=preprocess)

test_generator = ImageDataGenerator(rescale=1./255,
                                   preprocessing_function=preprocess)

In [ ]:
def augment_images(datagen, directory, batch_size, target_size, subset=None):
  augmented_imgs = datagen.flow_from_directory(directory = directory,
                                                    target_size = TARGET_SIZE,
                                                    batch_size = BATCH_SIZE,
                                                    subset = subset)
  return augmented_imgs

In [ ]:
train_aug = augment_images(train_generator, train_path, BATCH_SIZE, TARGET_SIZE, 'training')
valid_aug = augment_images(valid_generator, train_path, BATCH_SIZE, TARGET_SIZE, 'validation')
test_aug = augment_images(test_generator, test_path, BATCH_SIZE, TARGET_SIZE)

Found 2681 images belonging to 4 classes.
Found 667 images belonging to 4 classes.
Found 840 images belonging to 4 classes.


## Model Specification

In [ ]:
from tensorflow.keras.applications import VGG16, Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout  
from tensorflow.keras.optimizers import Adam

In [ ]:
vgg_model = VGG16(input_shape=(224, 224, 3), weights="imagenet", include_top=False)

In [ ]:
xception_model = Xception(input_shape=(224, 224, 3), weights="imagenet", include_top=False)

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
def build_model(pretrained_model):

  for layer in pretrained_model.layers:
    layer.trainable = False

  model = Sequential()
  model.add(pretrained_model)

  #Adding our own classifier
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate = LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def train_model(model, name):

  checkpoint = ModelCheckpoint(f'/content/drive/MyDrive/Omdena/Osun Nigeria/Model/weights_{name}.hdf5', monitor='val_loss', save_best_only=True)

  stop = EarlyStopping(monitor="val_loss", patience=4)

  history = model.fit(train_aug,
                    steps_per_epoch=len(train_aug)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=valid_aug,
                    verbose=2,
                    shuffle=True,
                    callbacks = [checkpoint])
  return history

In [ ]:
xcep_hist = train_model(build_model(xception_model), 'xception_rf')